In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("../Data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis = "columns", how = "all")
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
X = df.drop("koi_disposition", axis = 1)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [4]:
y = df["koi_disposition"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y)
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
6080,1,0,0,0,12.496435,2.213000e-04,-2.213000e-04,132.035800,0.014300,-0.014300,...,-286,3.805,0.390,-0.130,2.730,0.535,-1.248,289.23080,44.412483,13.054
3001,0,0,0,0,11.615625,1.528000e-04,-1.528000e-04,131.968430,0.008230,-0.008230,...,-72,4.083,0.368,-0.092,1.453,0.218,-0.510,293.52756,41.111439,15.162
570,0,1,0,0,10.980246,6.930000e-07,-6.930000e-07,137.137607,0.000053,-0.000053,...,-159,4.462,0.098,-0.182,0.897,0.238,-0.119,282.79764,43.578129,14.212
4897,1,0,0,0,466.908240,1.194000e-02,-1.194000e-02,136.373100,0.019000,-0.019000,...,-146,4.456,0.102,-0.361,0.867,0.448,-0.103,297.65436,43.178551,15.202
625,0,1,1,1,1.061933,1.250000e-06,-1.250000e-06,133.850441,0.000978,-0.000978,...,-167,3.975,0.259,-0.111,1.851,0.383,-0.575,288.90253,44.632992,12.953


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [6]:
# Scale X values
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Label Encode the target feature from human words to machine id's
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [8]:
print(X_train_scaled.shape, y_train_categorical.shape)

(5243, 40) (5243, 3)


# Tune Model Parameters

In [9]:
# Create the model and the layers
model = Sequential()
model.add(Dense(units = 50, activation = "relu", input_dim = X_train_scaled.shape[1]))
model.add(Dense(units = 100, activation = "relu"))
model.add(Dense(units = 150, activation = "relu"))
model.add(Dense(units = 200, activation = "relu"))
model.add(Dense(units = y_train_categorical.shape[1], activation = "softmax"))

In [10]:
# Compile the model
model.compile(optimizer = "adam",
              loss = "categorical_crossentropy",
              metrics = ["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2050      
                                                                 
 dense_1 (Dense)             (None, 100)               5100      
                                                                 
 dense_2 (Dense)             (None, 150)               15150     
                                                                 
 dense_3 (Dense)             (None, 200)               30200     
                                                                 
 dense_4 (Dense)             (None, 3)                 603       
                                                                 
Total params: 53,103
Trainable params: 53,103
Non-trainable params: 0
_________________________________________________________________


# Train Tuned Model

In [11]:
model.fit(X_train_scaled,
          y_train_categorical,
          epochs = 500,
          shuffle = True,
          verbose = 2)

Epoch 1/500
164/164 - 1s - loss: 0.4958 - accuracy: 0.7353 - 540ms/epoch - 3ms/step
Epoch 2/500
164/164 - 0s - loss: 0.3791 - accuracy: 0.8035 - 112ms/epoch - 683us/step
Epoch 3/500
164/164 - 0s - loss: 0.3742 - accuracy: 0.8072 - 110ms/epoch - 672us/step
Epoch 4/500
164/164 - 0s - loss: 0.3560 - accuracy: 0.8232 - 120ms/epoch - 734us/step
Epoch 5/500
164/164 - 0s - loss: 0.3503 - accuracy: 0.8278 - 214ms/epoch - 1ms/step
Epoch 6/500
164/164 - 0s - loss: 0.3418 - accuracy: 0.8348 - 123ms/epoch - 752us/step
Epoch 7/500
164/164 - 0s - loss: 0.3305 - accuracy: 0.8421 - 228ms/epoch - 1ms/step
Epoch 8/500
164/164 - 0s - loss: 0.3245 - accuracy: 0.8508 - 137ms/epoch - 834us/step
Epoch 9/500
164/164 - 0s - loss: 0.3183 - accuracy: 0.8549 - 129ms/epoch - 788us/step
Epoch 10/500
164/164 - 0s - loss: 0.3214 - accuracy: 0.8495 - 175ms/epoch - 1ms/step
Epoch 11/500
164/164 - 0s - loss: 0.3067 - accuracy: 0.8625 - 176ms/epoch - 1ms/step
Epoch 12/500
164/164 - 0s - loss: 0.3016 - accuracy: 0.8642 - 

# Quantify our Trained Model

In [12]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose = 2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

55/55 - 0s - loss: 0.8668 - accuracy: 0.8724 - 130ms/epoch - 2ms/step
Normal Neural Network - Loss: 0.866844117641449, Accuracy: 0.8724256157875061


# Save the Model

In [13]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
filename = "../Models/deep_learning_model.h5"
model.save(filename)